In [1]:
!pip install transformers==4.17
# !pip install optuna
!pip install datasets
!pip install torchinfo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 74.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 46.0 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.47.1
    Uninstalling transformers-4.47.1:
      Successfully uninstalled transformers-4.47.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 3.3.1 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.17.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [2]:
from transformers import Trainer, TrainingArguments, DistilBertTokenizer, DistilBertForSequenceClassification
import torch
import pandas as pd
import nltk
import re
import ast
import numpy as np
from torch.utils.data import Dataset
from sklearn.metrics import precision_score, recall_score, f1_score, precision_recall_fscore_support, accuracy_score
from sklearn.model_selection import train_test_split
import torch.nn as nn
# import optuna
import time
from torchinfo import summary
from tabulate import tabulate

**Dataset Utilization provided by NLBSE**

In [3]:
splits = {'python_train': 'data/python_train-00000-of-00001.parquet', 'python_test': 'data/python_test-00000-of-00001.parquet'}
train_df = pd.read_parquet("hf://datasets/NLBSE/nlbse25-code-comment-classification/" + splits["python_train"])
test_df = pd.read_parquet("hf://datasets/NLBSE/nlbse25-code-comment-classification/" + splits["python_test"])

display(train_df.head())
display(test_df.head())
display(train_df.shape)
display(test_df.shape)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


,index,class,comment_sentence,partition,combo,labels
0,1,AccessMixin,functionality.,0,functionality. | AccessMixin,"[0, 0, 0, 0, 1]"
1,8,Atomic,"when it s used as a decorator, call wraps the ...",0,"when it s used as a decorator, call wraps the ...","[1, 0, 0, 0, 0]"
2,10,Atomic,"when it s used as a context manager, enter cre...",0,"when it s used as a context manager, enter cre...","[1, 0, 0, 0, 0]"
3,12,Atomic,exit commits the transaction or releases the s...,0,exit commits the transaction or releases the s...,"[1, 0, 0, 0, 0]"
4,14,Atomic,it s possible to disable the creation of savep...,0,it s possible to disable the creation of savep...,"[1, 0, 0, 0, 0]"


,index,class,comment_sentence,partition,combo,labels
0,0,AccessMixin,abstract cbv mixin that gives access mixins th...,1,abstract cbv mixin that gives access mixins th...,"[0, 0, 0, 0, 1]"
1,2,AmbiguityError,more than one migration matches a name prefix.,1,more than one migration matches a name prefix....,"[0, 0, 0, 0, 1]"
2,3,AppConfigStub,stub of an appconfig.,1,stub of an appconfig. | AppConfigStub,"[0, 0, 0, 0, 1]"
3,4,AppConfigStub,only provides a label and a dict of models.,1,only provides a label and a dict of models. | ...,"[0, 1, 0, 0, 0]"
4,5,Archive,the external api class that encapsulates an ar...,1,the external api class that encapsulates an ar...,"[0, 0, 1, 0, 1]"


(1884, 6)

(406, 6)

In [4]:
def remove_punctuation_except_pipe(text):
    return re.sub(r"[^\w\s\|]", "", text)

# Apply the function to the 'combo' column
train_df['combo'] = train_df['combo'].apply(remove_punctuation_except_pipe)
test_df['combo'] = test_df['combo'].apply(remove_punctuation_except_pipe)

# Convert data to lowercase
train_str_df = train_df[["class", "comment_sentence", "combo"]].applymap(
    lambda x: x.lower() if isinstance(x, str) else str(x)
)
test_str_df = test_df[["class", "comment_sentence", "combo"]].applymap(
    lambda x: x.lower() if isinstance(x, str) else str(x)
)

train_df[["class", "comment_sentence", "combo"]] = train_str_df[["class", "comment_sentence", "combo"]]
test_df[["class", "comment_sentence", "combo"]] = test_str_df[["class", "comment_sentence", "combo"]]

display(train_df.head())
display(test_df.head())

<ipython-input-4-df4392ae2e99>:9: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  train_str_df = train_df[["class", "comment_sentence", "combo"]].applymap(
<ipython-input-4-df4392ae2e99>:12: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  test_str_df = test_df[["class", "comment_sentence", "combo"]].applymap(


,index,class,comment_sentence,partition,combo,labels
0,1,accessmixin,functionality.,0,functionality | accessmixin,"[0, 0, 0, 0, 1]"
1,8,atomic,"when it s used as a decorator, call wraps the ...",0,when it s used as a decorator call wraps the e...,"[1, 0, 0, 0, 0]"
2,10,atomic,"when it s used as a context manager, enter cre...",0,when it s used as a context manager enter crea...,"[1, 0, 0, 0, 0]"
3,12,atomic,exit commits the transaction or releases the s...,0,exit commits the transaction or releases the s...,"[1, 0, 0, 0, 0]"
4,14,atomic,it s possible to disable the creation of savep...,0,it s possible to disable the creation of savep...,"[1, 0, 0, 0, 0]"


,index,class,comment_sentence,partition,combo,labels
0,0,accessmixin,abstract cbv mixin that gives access mixins th...,1,abstract cbv mixin that gives access mixins th...,"[0, 0, 0, 0, 1]"
1,2,ambiguityerror,more than one migration matches a name prefix.,1,more than one migration matches a name prefix ...,"[0, 0, 0, 0, 1]"
2,3,appconfigstub,stub of an appconfig.,1,stub of an appconfig | appconfigstub,"[0, 0, 0, 0, 1]"
3,4,appconfigstub,only provides a label and a dict of models.,1,only provides a label and a dict of models | a...,"[0, 1, 0, 0, 0]"
4,5,archive,the external api class that encapsulates an ar...,1,the external api class that encapsulates an ar...,"[0, 0, 1, 0, 1]"


In [5]:
X = list(train_df['combo'])
y = list(train_df['labels'])

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)

In [6]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=5)

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/256M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:1439: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(resolved_archive_file, map

In [7]:
class TextClassificationDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, device="cpu"):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.device = device

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, index):
        text = self.texts[index]
        label = self.labels[index]

        encoded_text = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=256,
            padding='max_length',
            truncation=True,
            return_token_type_ids=False,
            return_attention_mask=True,
            return_tensors='pt',
        )

        input_ids = encoded_text['input_ids'].squeeze().to(self.device)
        attention_mask = encoded_text['attention_mask'].squeeze().to(self.device)
        label = torch.tensor(label, dtype=torch.float).to(self.device)

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': label
        }

train_dataset = TextClassificationDataset(X_train, y_train, tokenizer)
eval_dataset = TextClassificationDataset(X_val, y_val, tokenizer)

In [8]:
class OneHotTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")

        # Use BCEWithLogitsLoss for multi-label classification
        loss_fct = nn.BCEWithLogitsLoss()
        loss = loss_fct(logits, labels.float())

        return (loss, outputs) if return_outputs else loss

In [9]:
def compute_metrics(pred):
    logits = pred.predictions
    preds = (logits > 0).astype(int)

    labels = pred.label_ids

    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    accuracy = (preds == labels).mean()

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }


# Optional compute_metrics function to get the accuracy, precision, recall, f1 for each of the individual labels
# without averaging them

# from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# def compute_metrics(pred):
#     # Get predictions and apply sigmoid to get probabilities
#     logits = pred.predictions
#     preds = (logits > 0).astype(int)  # Threshold at 0 to get binary predictions

#     # Get true labels
#     labels = pred.label_ids

#     # Calculate precision, recall, and f1-score for each label without averaging
#     precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average=None)
#     # Calculate accuracy per label (how many times each label is correctly predicted)
#     accuracy_per_label = (preds == labels).mean(axis=0)

#     # Prepare a dictionary to store metrics per category
#     metrics = {
#         'accuracy_per_label': accuracy_per_label,
#         'precision_per_label': precision,
#         'recall_per_label': recall,
#         'f1_per_label': f1
#     }

#     # Optionally, if you want to calculate an overall accuracy:
#     overall_accuracy = (preds == labels).all(axis=1).mean()  # This gives the fraction of samples where all labels are correctly predicted
#     metrics['overall_accuracy'] = overall_accuracy

#     return metrics

In [10]:
# def objective(trial):
#     learning_rate = trial.suggest_loguniform("learning_rate", 1e-6, 5e-5)
#     batch_size = trial.suggest_categorical("batch_size", [4, 8, 16])
#     num_train_epochs = trial.suggest_int("num_train_epochs", 3, 10)
#     weight_decay = trial.suggest_loguniform("weight_decay", 1e-4, 0.1)

#     training_args = TrainingArguments(
#         output_dir="./results",
#         num_train_epochs=num_train_epochs,
#         per_device_train_batch_size=batch_size,
#         per_device_eval_batch_size=batch_size * 2,
#         warmup_steps=10,
#         weight_decay=weight_decay,
#         evaluation_strategy="epoch",
#         learning_rate=learning_rate,
#         gradient_accumulation_steps=2,
#         report_to="none",
#     )

#     trainer = OneHotTrainer(
#         model=model,
#         args=training_args,
#         train_dataset=train_dataset,
#         eval_dataset=eval_dataset,
#         compute_metrics=compute_metrics,
#     )

#     trainer.train()
#     eval_results = trainer.evaluate()

#     return eval_results["eval_f1"]

In [11]:
# study = optuna.create_study(direction="maximize")
# study.optimize(objective, n_trials=20)

# print("Best trial:")
# print(study.best_trial)

In [12]:
# best_hyperparams = study.best_trial.params

training_args = TrainingArguments(
    output_dir="./best_results",
    num_train_epochs=15,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16 * 2,
    learning_rate=3e-05,
    weight_decay=0.028779217423793482,
    warmup_steps=10,
    evaluation_strategy="epoch",
    gradient_accumulation_steps=2,
    report_to="none",
)


trainer = OneHotTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)
trainer.train()

/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1507
  Num Epochs = 15
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 2
  Total optimization steps = 705


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
0,No log,0.476029,0.786207,0.308642,0.004938,0.009721
1,No log,0.397912,0.843501,0.618385,0.333333,0.400174
2,No log,0.337334,0.874271,0.725117,0.525926,0.600645
3,No log,0.330347,0.866313,0.693915,0.540741,0.582141
4,No log,0.307480,0.876923,0.771686,0.614815,0.667262
5,No log,0.289274,0.889125,0.774517,0.661728,0.697157
6,No log,0.295629,0.887533,0.771124,0.696296,0.721640
7,No log,0.291203,0.897082,0.780221,0.718519,0.746150
8,No log,0.295224,0.896552,0.782061,0.718519,0.746699
9,No log,0.316251,0.890186,0.767333,0.703704,0.727155


***** Running Evaluation *****
  Num examples = 377
  Batch size = 32
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
***** Running Evaluation *****
  Num examples = 377
  Batch size = 32
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
***** Running Evaluation *****
  Num examples = 377
  Batch size = 32
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in la

TrainOutput(global_step=705, training_loss=0.18181327928042582, metrics={'train_runtime': 510.5277, 'train_samples_per_second': 44.278, 'train_steps_per_second': 1.381, 'total_flos': 1497094163573760.0, 'train_loss': 0.18181327928042582, 'epoch': 14.99})

In [13]:
!mkdir 'best_model_distilbert_python'
!mkdir 'best_model_distilbert_python_tokenizer'

# Save model and tokenizer
trainer.save_model('./best_model_distilbert_python')
tokenizer.save_pretrained('./best_model_distilbert_python_tokenizer')

# Load model and tokenizer
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

model = DistilBertForSequenceClassification.from_pretrained('./best_model_distilbert_python')
tokenizer = DistilBertTokenizer.from_pretrained('./best_model_distilbert_python_tokenizer')

Saving model checkpoint to ./best_model_distilbert_python
Configuration saved in ./best_model_distilbert_python/config.json
Model weights saved in ./best_model_distilbert_python/pytorch_model.bin
tokenizer config file saved in ./best_model_distilbert_python_tokenizer/tokenizer_config.json
Special tokens file saved in ./best_model_distilbert_python_tokenizer/special_tokens_map.json
loading configuration file ./best_model_distilbert_python/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4
  },
  "max_position_embeddings": 512,
 

## Model Testing

In [14]:
X_test = list(test_df['combo'])
y_test = list(test_df['labels'])

In [15]:
df_test = pd.DataFrame({"combo":X_test,"labels":y_test})
test_text = df_test.combo.values
test_label = df_test.labels.values

In [16]:
type(test_label[0])

numpy.ndarray

In [17]:
test_dataset = TextClassificationDataset(test_text, test_label, tokenizer)

# Measure inference time
start_time = time.time()

# Predict on the test dataset
predictions = trainer.predict(test_dataset)

# Calculate elapsed time
end_time = time.time()
inference_time = end_time - start_time

# Compute metrics using the `compute_metrics` function
metrics = compute_metrics(predictions)

# Display metrics and inference time
print("Evaluation Metrics:", metrics)
print(f"Inference Time: {inference_time:.2f} seconds")

***** Running Prediction *****
  Num examples = 406
  Batch size = 32


Evaluation Metrics: {'accuracy': 0.8566502463054187, 'precision': 0.6760321530081983, 'recall': 0.6238532110091743, 'f1': 0.647104064708335}
Inference Time: 3.42 seconds


In [18]:
!zip -r best_model_distilbert_python.zip './best_model_distilbert_python'
!zip -r best_model_distilbert_python_tokenizer.zip './best_model_distilbert_python_tokenizer'

# For downloading model locally
# from google.colab import files
# files.download('checkpoint-6000.zip')

  adding: best_model_distilbert_python/ (stored 0%)
  adding: best_model_distilbert_python/pytorch_model.bin (deflated 8%)
  adding: best_model_distilbert_python/config.json (deflated 51%)
  adding: best_model_distilbert_python/training_args.bin (deflated 51%)
  adding: best_model_distilbert_python_tokenizer/ (stored 0%)
  adding: best_model_distilbert_python_tokenizer/special_tokens_map.json (deflated 40%)
  adding: best_model_distilbert_python_tokenizer/tokenizer_config.json (deflated 43%)
  adding: best_model_distilbert_python_tokenizer/vocab.txt (deflated 53%)


In [19]:
# prompt: transfer the checkpoint-6000.zip created to the google drive

from google.colab import drive
drive.mount('/content/drive')

!cp best_model_distilbert_python.zip "/content/drive/MyDrive/nlbse/models/DistilBert_Python/"
!cp best_model_distilbert_python_tokenizer.zip "/content/drive/MyDrive/nlbse/models/DistilBert_Python_Tokenizer/"

Mounted at /content/drive
cp: cannot create regular file '/content/drive/MyDrive/nlbse/models/DistilBert_Python/': Not a directory
cp: cannot create regular file '/content/drive/MyDrive/nlbse/models/DistilBert_Python_Tokenizer/': Not a directory


In [20]:
# Save the tokenizer
# tokenizer.save_pretrained('/content/drive/MyDrive/FYP/Revised/Models/DistilBert_Python_Tokenizer/')

# **Load and Test Model**

In [21]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [22]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

# Replace 'path/to/checkpoint-folder' with the actual path to your checkpoint folder.
model_folder = '/content/drive/MyDrive/nlbse/models/DistilBert_Python/'
tokenizer_folder = '/content/drive/MyDrive/nlbse/models/DistilBert_Python_Tokenizer/'

!unzip '/content/drive/MyDrive/nlbse/models/DistilBert_Python/best_model_distilbert_python.zip' -d './'
!unzip '/content/drive/MyDrive/nlbse/models/DistilBert_Python_Tokenizer/best_model_distilbert_python_tokenizer.zip' -d './'

unzip:  cannot find or open /content/drive/MyDrive/nlbse/models/DistilBert_Python/best_model_distilbert_python.zip, /content/drive/MyDrive/nlbse/models/DistilBert_Python/best_model_distilbert_python.zip.zip or /content/drive/MyDrive/nlbse/models/DistilBert_Python/best_model_distilbert_python.zip.ZIP.
unzip:  cannot find or open /content/drive/MyDrive/nlbse/models/DistilBert_Python_Tokenizer/best_model_distilbert_python_tokenizer.zip, /content/drive/MyDrive/nlbse/models/DistilBert_Python_Tokenizer/best_model_distilbert_python_tokenizer.zip.zip or /content/drive/MyDrive/nlbse/models/DistilBert_Python_Tokenizer/best_model_distilbert_python_tokenizer.zip.ZIP.


In [23]:
best_model_distilbert_python = 'best_model_distilbert_python'
best_model_distilbert_python_tokenizer = 'best_model_distilbert_python_tokenizer'

In [24]:
# Load the model and tokenizer from the checkpoint
model = DistilBertForSequenceClassification.from_pretrained(best_model_distilbert_python)
tokenizer = DistilBertTokenizer.from_pretrained(best_model_distilbert_python_tokenizer)

text = "what is this?"
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)
output

loading configuration file best_model_distilbert_python/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "problem_type": "multi_label_classification",
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "torch_dtype": "float32",
  "transformers_version": "4.17.0",
  "vocab_size": 30522
}

loading weights file best_model_distilbert_python/pytorch

SequenceClassifierOutput(loss=None, logits=tensor([[-1.1510, -3.0466, -0.0238, -2.6515, -2.5161]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [25]:
logits = output.logits

# Apply sigmoid to convert logits to probabilities
probabilities = torch.sigmoid(logits)

# Define a threshold to determine if a class is positive
threshold = 0.5
predicted_indices = (probabilities > threshold).nonzero(as_tuple=True)[1]

# Convert indices to a list for output
predicted_classes = (predicted_indices + 1).tolist()

# Format the output to display classes
if len(predicted_classes) > 0:
    predicted_classes_str = ", ".join(map(str, predicted_classes))
    print(f"Predicted class/es: {predicted_classes_str}")
else:
    print("No positive classes predicted.")

No positive classes predicted.


In [26]:
# import torch

# # Logits from the model output
# logits =  output.logits #torch.tensor([[-17.5500, -13.8129, 9.7042, -16.6331, -6.6066]])

# # Get the index of the highest score
# predicted_class = ((torch.argmax(logits, dim=1).item()) + 1)

# print(f"Predicted class: {predicted_class}")
